# Adding a model to MLFlow

### Imports

In [5]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# For reading object store credentials
import dotenv
if dotenv.load_dotenv('../.env'):
    print("Successfully loaded .env file")
else:
    print("Loading .env file failed")

Successfully loaded .env file


### Setting up experiment and tracking url

In [6]:
mlflow.set_tracking_uri('http://localhost:5000')

experiment_name = "demo-iris"
mlflow.set_experiment(experiment_name)
print("Using experiment id:", mlflow.get_experiment_by_name(experiment_name).experiment_id)

Using experiment id: 1


### Train and add the model to MLFlow

In [7]:
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
with mlflow.start_run() as run:
    params = {"max_depth": 2, "random_state": 42}
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)

    # Infer the model signature
    y_pred = model.predict(X_test)
    signature = infer_signature(X_test, y_pred)

    # Log parameters and metrics using the MLflow APIs
    mlflow.log_params(params)
    mlflow.log_metrics({"mse": mean_squared_error(y_test, y_pred)})

    model_uri = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="random-forest",
        registered_model_name="IrisRandomForest"
    )

2025/03/26 12:43:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'IrisRandomForest'.
2025/03/26 12:43:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IrisRandomForest, version 1


🏃 View run classy-smelt-896 at: http://localhost:5000/#/experiments/1/runs/d6bb3069c6b1431bb99a56f370ff76f0
🧪 View experiment at: http://localhost:5000/#/experiments/1


Created version '1' of model 'IrisRandomForest'.


### Loading the model and inference

In [8]:
model_loaded = mlflow.pyfunc.load_model("models:/IrisRandomForest/latest")
print("Loaded model predicts:", model_loaded.predict(X_test[:5]))

Loaded model predicts: [-25.73795219 -19.61227378  41.662821   -39.30386685 -32.70582446]
